In [13]:
import pandas as pd

# Cargar datos de normalización
hospital_df = pd.read_csv('../downloads/normalizacion/hospital.csv', delimiter=',')
hospital_activity_df = pd.read_csv('../downloads/normalizacion/hospital_activity.csv', delimiter=';')
hospital_municipality_df = pd.read_csv('../downloads/normalizacion/hospital_municipality.csv', delimiter=';')
hospital_resources_df = pd.read_csv('../downloads/normalizacion/hospital_resources.csv', delimiter=';')
municipality_df = pd.read_csv('../downloads/normalizacion/municipality.csv', delimiter=',')
municipality_demographics_df = pd.read_csv('../downloads/normalizacion/municipality_demographics.csv', delimiter=',')

# Vista previa de los datos
print('Hospitales:')
display(hospital_df.head())
print('Actividad hospitalaria:')
display(hospital_activity_df.head())
print('Hospital-Municipio:')
display(hospital_municipality_df.head())
print('Recursos hospitalarios:')
display(hospital_resources_df.head())
print('Municipios:')
display(municipality_df.head())
print('Demografía municipios:')
display(municipality_demographics_df.head())

# Convert merge keys to string for safe merging
hospital_municipality_df['municipality_id'] = hospital_municipality_df['municipality_id'].astype(str)
municipality_df['id'] = municipality_df['id'].astype(str)
hospital_municipality_df['hospital_id'] = hospital_municipality_df['hospital_id'].astype(str)
hospital_df['id'] = hospital_df['id'].astype(str)

# Relacionar municipio con hospital usando hospital-municipio como pivote
relacion_df = hospital_municipality_df.merge(municipality_df, left_on='municipality_id', right_on='id', how='left')
relacion_df = relacion_df.merge(hospital_df, left_on='hospital_id', right_on='id', how='left', suffixes=('_municipio', '_hospital'))

# Vista previa de la relación municipio-hospital
print('Relación Municipio-Hospital:')
display(relacion_df.head())

# Filtrar recursos hospitalarios solo para el año 2022 (usando .copy() para evitar SettingWithCopyWarning)
hospital_resources_2022 = hospital_resources_df[hospital_resources_df['year'].astype(str) == '2022'].copy()
hospital_resources_2022['hospital_id'] = hospital_resources_2022['hospital_id'].astype(str)

# Unir los recursos hospitalarios (año 2022) a la relación municipio-hospital
relacion_recursos_df = relacion_df.merge(hospital_resources_2022, on='hospital_id', how='left', suffixes=('', '_recursos'))

# Vista previa de la relación municipio-hospital con recursos hospitalarios 2022
print('Relación Municipio-Hospital con Recursos Hospitalarios (2022):')
display(relacion_recursos_df.head())

# Filtrar actividad hospitalaria solo para el año 2022 (usando .copy() para evitar SettingWithCopyWarning)
hospital_activity_2022 = hospital_activity_df[hospital_activity_df['year'].astype(str) == '2022'].copy()
hospital_activity_2022['hospital_id'] = hospital_activity_2022['hospital_id'].astype(str)

# Unir la actividad hospitalaria (año 2022) a la relación municipio-hospital con recursos
relacion_full_df = relacion_recursos_df.merge(hospital_activity_2022, on='hospital_id', how='left', suffixes=('', '_actividad'))

# Vista previa de la relación municipio-hospital con recursos y actividad hospitalaria 2022
print('Relación Municipio-Hospital con Recursos y Actividad Hospitalaria (2022):')
display(relacion_full_df.head(5))

# Filtrar demografía municipios solo para el año 2022 (usando .copy() para evitar SettingWithCopyWarning)
municipality_demographics_2022 = municipality_demographics_df[municipality_demographics_df['year'].astype(str) == '2022'].copy()
municipality_demographics_2022['id_secondary_municipality'] = municipality_demographics_2022['id_secondary_municipality'].astype(str)

# Convertir id_secondary a string para asegurar el merge
relacion_full_df['id_secondary'] = relacion_full_df['id_secondary'].astype(str)

# Unir la demografía municipios (año 2022) a la relación completa usando id_secondary <-> id_secondary_municipality
relacion_full_demografia_df = relacion_full_df.merge(municipality_demographics_2022, left_on='id_secondary', right_on='id_secondary_municipality', how='left', suffixes=('', '_demografia'))

# Vista previa de la relación municipio-hospital con recursos, actividad y demografía 2022
print('Relación Municipio-Hospital con Recursos, Actividad y Demografía (2022):')
display(relacion_full_demografia_df.head(10))


Hospitales:


,id,name_large,name,id_municipality,utm_x,utm_y,tag,address,url,id_via,name_municipality,latitude,longitude
0,280724,Hospital Centro Sanitario de Vida y Esperanza....,HOSPITAL CENTRO SANITARIO DE VIDA Y ESPERANZA,28079,436745,4471110,Hospital Centro Sanitario de Vida y Esperanza,"Glorieta del Ejército, s/n",https://centrossanitarios.sanidadmadrid.org/Re...,07901901,Madrid,40.388185,-3.745275
1,CH0001,Fremap Hospital y Centro de Rehabilitación de ...,FREMAP HOSPITAL Y CENTRO DE REHABILITACIÓN DE ...,28080,427799,4478345,Fremap Hospital y Centro de Rehabilitación de ...,"Carretera de Pozuelo, 61",https://www.fremap.es,08000170,Majadahonda,40.452632,-3.851488
2,CH0004,Centro San Juan de Dios. Avenida de San Juan d...,CENTRO SAN JUAN DE DIOS,28040,447682,4445737,Centro San Juan de Dios,"Avenida de San Juan de Dios, 1",https://sjd.es/location/centro-san-juan-de-dio...,040A0157,Ciempozuelos,40.160362,-3.614350
3,CH0006,Fuensanta S.l. (Clínica Fuensanta). Calle de A...,FUENSANTA S.L. (CLINICA FUENSANTA),28079,445713,4476806,Fuensanta S.l. (Clínica Fuensanta),"Calle de Arturo Soria, 17",https://www.viamedsalud.com/hospital-fuensanta/,07900582,Madrid,40.440131,-3.640106
4,CH0008,"Clínica La Luz, S.l.. Calle del General Rodrig...","CLINICA LA LUZ, S.L.",28079,439451,4477308,"Clínica La Luz, S.l.","Calle del General Rodrigo, 8",https://www.quironsalud.com/es/red-centros/hos...,07909065,Madrid,40.444221,-3.713985


Actividad hospitalaria:


,hospital_id,year,type_activity,total
0,CH0096,2012,Estancia media global,5.72
1,CH0096,2013,Estancia media global,6.08
2,CH0096,2014,Estancia media global,6.03
3,CH0096,2015,Estancia media global,6.22
4,CH0096,2016,Estancia media global,6.16


Hospital-Municipio:


...

,hospital_id,municipality_id,id_municipio,id_secondary,name_municipio,tag_municipio,id_hospital,name_large,name_hospital,id_municipality,...,tag_hospital,address,url,id_via,name_municipality,latitude,longitude,year,type_resources,total
0,hospital_central_de_la_defensa_gomez_ulla,28079,28079,796.0,Madrid,Madrid,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022.0,Camas instaladas,475.0
1,CH0053,28079,28079,796.0,Madrid,Madrid,CH0053,Hospital Clínico San Carlos. Calle del Profeso...,HHOSPITAL CLINICO SAN CARLOS,28079.0,...,Hospital Clínico San Carlos,"Calle del Profesor Martín Lagos, s/n",https://www.comunidad.madrid/hospital/clinicos...,07911286,Madrid,40.43987,-3.719728,2022.0,Camas instaladas,861.0
2,CH0027,28018,28018,186.0,Becerril de la Sierra,Becerril de la Sierra,CH0027,Hospital de El Escorial. Carretera de Guadarra...,HOSPITAL DE EL ESCORIAL,28131.0,...,Hospital de El Escorial,Carretera de Guadarrama-San Lorenzo del Escori...,https://www.comunidad.madrid/hospital/elescorial/,13100155,San Lorenzo de El Escorial,40.60633,-4.118784,2022.0,Camas instaladas,92.0
3,CH0027,28038,28038,382.0,Cercedilla,Cercedilla,CH0027,Hospital de El Escorial. Carretera de Guadarra...,HOSPITAL DE EL ESCORIAL,28131.0,...,Hospital de El Escorial,Carretera de Guadarrama-San Lorenzo del Escori...,https://www.comunidad.madrid/hospital/elescorial/,13100155,San Lorenzo de El Escorial,40.60633,-4.118784,2022.0,Camas instaladas,92.0
4,CH0027,28046,28046,468.0,Collado Mediano,Collado Mediano,CH0027,Hospital de El Escorial. Carretera de Guadarra...,HOSPITAL DE EL ESCORIAL,28131.0,...,Hospital de El Escorial,Carretera de Guadarrama-San Lorenzo del Escori...,https://www.comunidad.madrid/hospital/elescorial/,13100155,San Lorenzo de El Escorial,40.60633,-4.118784,2022.0,Camas instaladas,92.0


Relación Municipio-Hospital con Recursos y Actividad Hospitalaria (2022):


,hospital_id,municipality_id,id_municipio,id_secondary,name_municipio,tag_municipio,id_hospital,name_large,name_hospital,id_municipality,...,id_via,name_municipality,latitude,longitude,year,type_resources,total,year_actividad,type_activity,total_actividad
0,hospital_central_de_la_defensa_gomez_ulla,28079,28079,796.0,Madrid,Madrid,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2022.0,Camas instaladas,475.0,2022.0,Estancia media global,7.98
1,hospital_central_de_la_defensa_gomez_ulla,28079,28079,796.0,Madrid,Madrid,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2022.0,Camas instaladas,475.0,2022.0,Total ingresos,9339.00
2,hospital_central_de_la_defensa_gomez_ulla,28079,28079,796.0,Madrid,Madrid,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2022.0,Camas instaladas,475.0,2022.0,Ingresos urgentes,5122.00
3,hospital_central_de_la_defensa_gomez_ulla,28079,28079,796.0,Madrid,Madrid,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2022.0,Camas instaladas,475.0,2022.0,Ingresos programados,4217.00
4,hospital_central_de_la_defensa_gomez_ulla,28079,28079,796.0,Madrid,Madrid,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2022.0,Camas instaladas,475.0,2022.0,Urgencias totales,84513.00


Relación Municipio-Hospital con Recursos, Actividad y Demografía (2022):


,hospital_id,municipality_id,id_municipio,id_secondary,name_municipio,tag_municipio,id_hospital,name_large,name_hospital,id_municipality,...,type_resources,total,year_actividad,type_activity,total_actividad,id_secondary_municipality,name,total_demografia,year_demografia,range
0,hospital_central_de_la_defensa_gomez_ulla,28079,28079,796.0,Madrid,Madrid,NaN,NaN,NaN,NaN,...,Camas instaladas,475.0,2022.0,Estancia media global,7.98,NaN,NaN,NaN,NaN,NaN
1,hospital_central_de_la_defensa_gomez_ulla,28079,28079,796.0,Madrid,Madrid,NaN,NaN,NaN,NaN,...,Camas instaladas,475.0,2022.0,Total ingresos,9339.00,NaN,NaN,NaN,NaN,NaN
2,hospital_central_de_la_defensa_gomez_ulla,28079,28079,796.0,Madrid,Madrid,NaN,NaN,NaN,NaN,...,Camas instaladas,475.0,2022.0,Ingresos urgentes,5122.00,NaN,NaN,NaN,NaN,NaN
3,hospital_central_de_la_defensa_gomez_ulla,28079,28079,796.0,Madrid,Madrid,NaN,NaN,NaN,NaN,...,Camas instaladas,475.0,2022.0,Ingresos programados,4217.00,NaN,NaN,NaN,NaN,NaN
4,hospital_central_de_la_defensa_gomez_ulla,28079,28079,796.0,Madrid,Madrid,NaN,NaN,NaN,NaN,...,Camas instaladas,475.0,2022.0,Urgencias totales,84513.00,NaN,NaN,NaN,NaN,NaN
5,CH0053,28079,28079,796.0,Madrid,Madrid,CH0053,Hospital Clínico San Carlos. Calle del Profeso...,HHOSPITAL CLINICO SAN CARLOS,28079.0,...,Camas instaladas,861.0,2022.0,Estancia media global,7.53,NaN,NaN,NaN,NaN,NaN
6,CH0053,28079,28079,796.0,Madrid,Madrid,CH0053,Hospital Clínico San Carlos. Calle del Profeso...,HHOSPITAL CLINICO SAN CARLOS,28079.0,...,Camas instaladas,861.0,2022.0,Total ingresos,28705.00,NaN,NaN,NaN,NaN,NaN
7,CH0053,28079,28079,796.0,Madrid,Madrid,CH0053,Hospital Clínico San Carlos. Calle del Profeso...,HHOSPITAL CLINICO SAN CARLOS,28079.0,...,Camas instaladas,861.0,2022.0,Ingresos urgentes,20254.00,NaN,NaN,NaN,NaN,NaN
8,CH0053,28079,28079,796.0,Madrid,Madrid,CH0053,Hospital Clínico San Carlos. Calle del Profeso...,HHOSPITAL CLINICO SAN CARLOS,28079.0,...,Camas instaladas,861.0,2022.0,Ingresos programados,8451.00,NaN,NaN,NaN,NaN,NaN
9,CH0053,28079,28079,796.0,Madrid,Madrid,CH0053,Hospital Clínico San Carlos. Calle del Profeso...,HHOSPITAL CLINICO SAN CARLOS,28079.0,...,Camas instaladas,861.0,2022.0,Urgencias totales,145819.00,NaN,NaN,NaN,NaN,NaN
